In [43]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from utils.classifications_utils import *
from utils.data_processing_utils_test import *
from utils.data_visualization_utils import *
from utils.metrics_utils import *
from utils.grid_search_utils import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [44]:
no_wind_data = load_data(0, 4, 'project3', 'drone2')
temp = no_wind_data # temp is a Dataframe
no_wind_data = separate_data_based_on_apparatus(no_wind_data)
# no_wind_data is a dict

level_1_wind = load_data(1, 4, 'project3', 'drone2')
level_1_wind = separate_data_based_on_apparatus(level_1_wind)

In [45]:
acc_no_wind, gyro_no_wind, _, _ = no_wind_data.values()
label_0 = [0 for x in range(acc_no_wind.shape[0])]
# acc_no_wind is a Dataframe

acc_level_1_wind, gyro_level_1_wind, _, _ = level_1_wind.values()
label_1 = [1 for x in range(acc_level_1_wind.shape[0])]

# Split No Wind Data

In [46]:
a = 1500
b = 2000

In [47]:
X_train_acc_0, X_test_acc_0, y_train_acc_0, y_test_acc_0 = \
train_test_split(acc_no_wind, label_0, test_size=0.25, shuffle=False)

X_train_gyro_0, X_test_gyro_0, y_train_gyro_0, y_test_gyro_0 = \
train_test_split(gyro_no_wind, label_0, test_size=0.25, shuffle=False)

In [48]:
print(len(y_test_acc_0))

2398


In [49]:
#X_test_acc_0 = X_test_acc_0.iloc[a:b]
#X_test_gyro_0 = X_test_gyro_0.iloc[a:b]

In [50]:
#y_test_acc_0 = y_test_acc_0[a:b]
#y_test_gyro_0 = y_test_gyro_0[a:b]

# Split Level 1 Wind

In [51]:
X_train_acc_1, X_test_acc_1, y_train_acc_1, y_test_acc_1 = \
train_test_split(acc_level_1_wind, label_1, test_size=0.25, shuffle=False)

X_train_gyro_1, X_test_gyro_1, y_train_gyro_1, y_test_gyro_1 = \
train_test_split(gyro_level_1_wind, label_1, test_size=0.25, shuffle=False)

In [52]:
#X_test_acc_1 = X_test_acc_1.iloc[a:b]
#X_test_gyro_1 = X_test_gyro_1.iloc[a:b]

In [53]:
#y_test_acc_1 = y_test_acc_1[a:b]
#y_test_gyro_1 = y_test_gyro_1[a:b]

# Stack data up

# Gyro

In [54]:
X_train_gyro = X_train_gyro_0.append(X_train_gyro_1)
#X_train_gyro = X_train_gyro.append(X_train_gyro_2)
#X_train_gyro = X_train_gyro.append(X_train_gyro_3)

#y_train_gyro = np.hstack((y_train_gyro_0, y_train_gyro_1, y_train_gyro_2, y_train_gyro_3))
y_train_gyro = np.hstack((y_train_gyro_0, y_train_gyro_1))

In [55]:
print(len(y_train_gyro))

14394


In [56]:
X_test_gyro = X_test_gyro_0.append(X_test_gyro_1)
#X_test_gyro = X_test_gyro.append(X_test_gyro_2)
#X_test_gyro = X_test_gyro.append(X_test_gyro_3)

#y_test_gyro = np.hstack((y_test_gyro_0, y_test_gyro_1, y_test_gyro_2, y_test_gyro_3))
y_test_gyro = np.hstack((y_test_gyro_0, y_test_gyro_1))

In [57]:
print(len(y_test_gyro))

4798


# Accelerometer

In [58]:
X_train_acc = X_train_acc_0.append(X_train_acc_1)
#X_train_acc = X_train_acc.append(X_train_acc_2)
#X_train_acc = X_train_acc.append(X_train_acc_3)

#y_train_acc = np.hstack((y_train_acc_0, y_train_acc_1, y_train_acc_2, y_train_acc_3))
y_train_acc = np.hstack((y_train_acc_0, y_train_acc_1))

In [59]:
X_test_acc = X_test_acc_0.append(X_test_acc_1)
#X_test_acc = X_test_acc.append(X_test_acc_2)
#X_test_acc = X_test_acc.append(X_test_acc_3)

#y_test_acc = np.hstack((y_test_acc_0, y_test_acc_1, y_test_acc_2, y_test_acc_3))
y_test_acc = np.hstack((y_test_acc_0, y_test_acc_1))

# Transform and generate features

In [60]:
from transformers.featureGenerator import FeatureGenerator

# Gyro

In [61]:
feature_generator_gyro = FeatureGenerator(1, 'gyro')
feature_generator_gyro.fit(X_train_gyro, 2)

FeatureGenerator(k=0, reduce_noise=False, sensor='gyro', sliding_window=1)

In [62]:
X_train_gyro = feature_generator_gyro.transform(X_train_gyro)

In [63]:
y_train_gyro = adjust_label_amount(y_train_gyro, 2)

In [64]:
feature_generator_gyro_test = FeatureGenerator(1, 'gyro')
feature_generator_gyro_test.fit(X_test_gyro, 2)

FeatureGenerator(k=0, reduce_noise=False, sensor='gyro', sliding_window=1)

In [65]:
X_test_gyro = feature_generator_gyro_test.transform(X_test_gyro)
y_test_gyro = adjust_label_amount(y_test_gyro, 2)

In [66]:
print(len(y_test_gyro))

4602


# Acc

In [67]:
feature_generator_acc = FeatureGenerator(1, 'acc')
feature_generator_acc.fit(X_train_acc, 2)

FeatureGenerator(k=0, reduce_noise=False, sensor='acc', sliding_window=1)

In [68]:
X_train_acc = feature_generator_acc.transform(X_train_acc)
y_train_acc = adjust_label_amount(y_train_acc, 2)

In [69]:
feature_generator_acc_test = FeatureGenerator(1, 'acc')
feature_generator_acc_test.fit(X_test_acc, 2)

FeatureGenerator(k=0, reduce_noise=False, sensor='acc', sliding_window=1)

In [70]:
X_test_acc = feature_generator_acc_test.transform(X_test_acc)
y_test_acc = adjust_label_amount(y_test_acc, 2)

# Train a random forest classifier and evaluate 

In [71]:
from sklearn.ensemble import GradientBoostingClassifier

# Gyro

In [72]:
clf_gyro = GradientBoostingClassifier()
clf_gyro.fit(X_train_gyro, y_train_gyro)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

# Pickle trained classifier

Enabled only when it is necessary

In [73]:
y_pred_gyro = clf_gyro.predict(X_test_gyro)

# Confusion Matrix

In [74]:
get_confusion_matrix(y_test_gyro, y_pred_gyro)

,actual_0,actual_1
predicted_0,1989,312
predicted_1,347,1954


# Accuracy

In [75]:
clf_gyro.score(X_test_gyro, y_test_gyro)

0.8568013906996957

# Acc

In [76]:
clf_acc = GradientBoostingClassifier()
clf_acc.fit(X_train_acc, y_train_acc)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

# Pickle trained classifiers

Enabled only when it is necessary

joblib.dump(clf_acc, "pickled_models/random_forest_acc.pkl")

In [77]:
y_pred_acc = clf_acc.predict(X_test_acc)

# Confusion Matrix

In [78]:
get_confusion_matrix(y_test_acc, y_pred_acc)

,actual_0,actual_1
predicted_0,1942,359
predicted_1,282,2019


In [79]:
clf_acc.score(X_test_acc, y_test_acc)

0.8607127335940895